In [ ]:
# Imports

import pandas as pd
import tensorflow as tf
import os
import numpy as np
from sklearn.decomposition import PCA
from keras.utils import load_img, img_to_array, normalize
from keras.applications import ResNet50, Xception
from keras.layers import GlobalMaxPooling2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Input, Layer
from numpy.linalg import norm
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tqdm import tqdm
import pickle
from keras.optimizers import Adam
#tf.keras.applications.mobilenet_v2.MobileNetV2
#tf.keras.applications.xception.Xception
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg
tf.__version__

In [ ]:
# Note: Add Directory address according to you
MAIN_DIR = 'C:/project/fashion/static/myntradataset/'
df = pd.read_csv(MAIN_DIR+'styles.csv', on_bad_lines='skip')
df.head()

In [ ]:
df['image'] = df['id'].astype(str) + '.jpg'
df.head()

In [ ]:
image_files = os.listdir(MAIN_DIR+'images')
df['present'] = df['image'].apply(lambda x: x in image_files)
df.head()

In [ ]:
df['present'].value_counts()

In [ ]:
df = df[df['present']].reset_index(drop=True)
df.shape

In [ ]:
# if processing all images is not possible, use only images of clothes, you can do so by using following line
# styles = df[df['masterCategory'].isin(['Apparel'])]
styles = df
styles.head()

In [ ]:
# Paths to your images
image_paths = ['C:/project/fashion/static/myntradataset/images/{}.jpg'.format(id) for id in styles['id']]
image_paths[0:5]

In [ ]:
# Pre trained model
base_model = tf.keras.applications.xception.Xception(weights='imagenet',
                      include_top=False,
                      input_shape = (224, 224, 3))
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    GlobalMaxPooling2D()
])

model.summary()

In [ ]:
# Feature Extraction
def feature_extraction(img_path, model):
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = tf.keras.applications.xception.preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result

In [ ]:
# Collect all features
feature_list = []

for file in image_paths:
    feature_list.append(feature_extraction(file, model))

In [ ]:
pickle.dump(feature_list, open('xception_features.pkl', 'wb')) 

In [ ]:
# Create list of images and save it in pickle file, so Flask web app can call it directly
pc_paths = ['{}.jpg'.format(id) for id in styles['id']]
pc_paths[0:5]

In [ ]:
pickle.dump(pc_paths, open('filenamesPC.pkl', 'wb'))

In [ ]:
print(len(image_paths))